In [ ]:
import torch
from diffusers import WanPipeline
from diffusers.utils import export_to_video, load_video
from vibt.wan import load_vibt_weight, encode_video
from vibt.scheduler import ViBTScheduler

In [ ]:
base_model_id = "Wan-AI/Wan2.1-T2V-1.3B-Diffusers"
pipe = WanPipeline.from_pretrained(base_model_id, torch_dtype=torch.bfloat16).to("cuda")

In [ ]:
load_vibt_weight(
    pipe.transformer,
    "Yuanshi/ViBT",
    "video/video_colorization.safetensors",
)
pipe.scheduler = ViBTScheduler.from_scheduler(pipe.scheduler)

In [ ]:
source_video = load_video("../assets/video_00000106.mp4")
gray_video = [each.convert("L").convert("RGB") for each in source_video]
source_latents = encode_video(pipe, gray_video)
prompt = ""


pipe.scheduler.set_parameters(noise_scale=1.0, shift_gamma=5.0, seed=42)

output = pipe(
    prompt=prompt,
    num_inference_steps=20,
    guidance_scale=1.0,
    latents=source_latents,
).frames[0]
export_to_video(output, "video_106_coloring.mp4", fps=15)